In [24]:
import pandas as pd
from rich import print, pretty
from rich.console import Console
from icecream import ic
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances_argmin as distances_argmin
from embed_sources import create_sentence_embedding
from sentence_transformers import SentenceTransformer

console = Console()
pretty.install()

In [4]:
filepath = Path.cwd().joinpath("embeddings", "so_database_app.csv")
so_df = pd.read_csv(filepath)
so_df.head()

,input_text,output_text,category
0,"python's inspect.getfile returns ""<string>""<p>...",<p><code>&lt;string&gt;</code> means that the ...,python
1,Passing parameter to function while multithrea...,<p>Try this and note the difference:</p>\n<pre...,python
2,How do we test a specific method written in a ...,"<p>Duplicate of <a href=""https://stackoverflow...",python
3,how can i remove the black bg color of an imag...,<p>The alpha channel &quot;disappears&quot; be...,python
4,How to extract each sheet within an Excel file...,<p>You need to specify the <code>index</code> ...,python


In [9]:
console.print(f"Shape of the data is [bold green underline]{so_df.shape}", style="bold yellow")

Shape of the data is (2000, 3)

In [16]:
filepath = Path.cwd().joinpath("embeddings", "question_embeddings_app.pkl")
with open(filepath, "rb") as file:
    questions_embeddings = pickle.load(file)
print(questions_embeddings.shape)

# added the embedding to the dataframe
# This is acting like our vector DB
so_df["embeddings"] = questions_embeddings.tolist() 

(2000, 768)

In [33]:
# Now lets ask the LLM a question
query = ['How to concat dataframes pandas']

model = SentenceTransformer("all-mpnet-base-v2")

# get the embedding of the query
query_embedding, _ = create_sentence_embedding(query, model, bert = False)
print(query_embedding.shape)
cos_similarity = cosine_similarity(query_embedding.tolist(),
                            list(so_df.embeddings.values))

print(cos_similarity.shape)

# find the embedding with the highest similarity value
index_doc_cosine = np.argmax(cos_similarity)
print(index_doc_cosine)

# get the distances
index_doc_distances = distances_argmin(
                    query_embedding.tolist(),
                    list(so_df.embeddings.values))[0]


PCA of an array of one sample does not make sense.
So returning the full array.

(1, 768)

(1, 2000)

1073

"matching very long and complex version numbers with regular expressions in python<p>I have lots of lines like this.</p>\n<pre><code>some text some more text v3.1.0-beta.4 more &amp; more text\nsome text some v2 build.3 some more text more &amp; more text\nsome text some v21.1.23456.551436a4 alpha.4 some more text v16.1.2 more &amp; more text\n</code></pre>\n<p>version numbers are:</p>\n<pre><code>v3.1.0-beta.4\nv2 build 3\nv21.1.23456.551436a4 alpha.4 and v16.1.2\n</code></pre>\n<p>The problem is I can have have words beta or build or neither in my version numbers. variable length of version numbers.</p>\n<p>Here's what I have now and this can only match numbers.</p>\n<pre><code>\\d+(?:\\.\\d+)+\n</code></pre>\n<p>How do I search for the existence of one or more version numbers in a give line like shown in above lines like above?</p>"

In [35]:
# get the input text from the database using the above index
print(so_df.input_text[index_doc_cosine])

matching very long and complex version numbers with regular expressions in python<p>I have lots of lines like 
this.</p>
<pre><code>some text some more text v3.1.0-beta.4 more &amp; more text
some text some v2 build.3 some more text more &amp; more text
some text some v21.1.23456.551436a4 alpha.4 some more text v16.1.2 more &amp; more text
</code></pre>
<p>version numbers are:</p>
<pre><code>v3.1.0-beta.4
v2 build 3
v21.1.23456.551436a4 alpha.4 and v16.1.2
</code></pre>
<p>The problem is I can have have words beta or build or neither in my version numbers. variable length of version 
numbers.</p>
<p>Here's what I have now and this can only match numbers.</p>
<pre><code>\d+(?:\.\d+)+
</code></pre>
<p>How do I search for the existence of one or more version numbers in a give line like shown in above lines like 
above?</p>

In [37]:
print(so_df.output_text[index_doc_cosine])

<p>You may try this regex:</p>
<pre><code>\bv[\d+]+[\w.]*(?:[-\s]+(?:alpha|beta|build)[\w.]*)?
</code></pre>
<p><a href="https://regex101.com/r/sualem/3/" rel="nofollow noreferrer">RegEx Demo</a></p>
<p><strong>Pattern Details:</strong></p>
<ul>
<li><code>\bv</code>: Match <code>v</code> after a word boundary</li>
<li><code>[\d+]+</code>: Match 1+ digit or dot characters</li>
<li><code>[\w.]*</code>: Match 0 or more word or dot characters</li>
<li><code>(?:[-\s]+(?:alpha|beta|build)[\w.]*)?</code>: starting with whitespace or hyphen, optionally match 
<code>alpha|beta|build</code> part followed by 0 or more word or dot characters</li>
</ul>

In [39]:
print(index_doc_distances)

1073